1. Необходимые функции

In [1]:
import pandas as pd
import json

def read_file(file_name):
    return pd.read_csv(file_name, compression="zip")
    # return next()


def get_memory_stat_by_column(df):
    memory_usage_stat = df.memory_usage(deep=True)
    total_memory_usage = memory_usage_stat.sum()
    print(f"file in memory size = {total_memory_usage // 1024:10} КБ")
    column_stat = list()
    for key in df.dtypes.keys():
        column_stat.append({
            "column_name": key,
            "memory_abs": memory_usage_stat[key] // 1024,
            "memory_per": round(memory_usage_stat[key] / total_memory_usage * 100, 4),
            "dtype": df.dtypes[key]
        })
    column_stat.sort(key=lambda x: x['memory_abs'], reverse=True)
    for column in column_stat:
        print(
            f"{column['column_name']:30}: {column['memory_abs']:10} КБ: {column['memory_per']:10}% : {column['dtype']}")


def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:  # предположим, что если это не датафрейм, то серия
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2  # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)


def opt_obj(df):
    converted_obj = pd.DataFrame()
    dataset_obj = df.select_dtypes(include=['object']).copy()

    for col in dataset_obj.columns:
        num_unique_values = len(dataset_obj[col].unique())
        num_total_values = len(dataset_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:, col] = dataset_obj[col].astype('category')
        else:
            converted_obj.loc[:, col] = dataset_obj[col]

    print(mem_usage(dataset_obj))
    print(mem_usage(converted_obj))
    return converted_obj


def opt_int(df):
    dataset_int = df.select_dtypes(include=['int'])
    """
    downcast:
            - 'integer' or 'signed': smallest signed int dtype (min.: np.int8)
            - 'unsigned': smallest unsigned int dtype (min.: np.uint8)
            - 'float': smallest float dtype (min.: np.float32)
    """
    converted_int = dataset_int.apply(pd.to_numeric, downcast='unsigned')
    print(mem_usage(dataset_int))
    print(mem_usage(converted_int))
    #
    compare_ints = pd.concat([dataset_int.dtypes, converted_int.dtypes], axis=1)
    compare_ints.columns = ['before', 'after']
    compare_ints.apply(pd.Series.value_counts)
    print(compare_ints)

    return converted_int


def opt_float(df):
    # # =======================================================================
    # # выполняем понижающее преобразование
    # # для столбцов типа float
    dataset_float = df.select_dtypes(include=['float'])
    converted_float = dataset_float.apply(pd.to_numeric, downcast='float')

    print(mem_usage(dataset_float))
    print(mem_usage(converted_float))

    compare_floats = pd.concat([dataset_float.dtypes, converted_float.dtypes], axis=1)
    compare_floats.columns = ['before', 'after']
    compare_floats.apply(pd.Series.value_counts)
    print(compare_floats)

    return converted_float



In [5]:
# steps 1-3
file_name = "c:/Users/Meow/Desktop/Lab6/agricultural_production_census_2002.zip"
dataset = read_file(file_name)
get_memory_stat_by_column(dataset)

# steps 4-6
optimized_dataset = dataset.copy()

converted_obj = opt_obj(dataset)
converted_int = opt_int(dataset)
converted_float = opt_float(dataset)
#
optimized_dataset[converted_obj.columns] = converted_obj
optimized_dataset[converted_int.columns] = converted_int
optimized_dataset[converted_float.columns] = converted_float

# 7
get_memory_stat_by_column(dataset)
print(mem_usage(dataset))
print(mem_usage(optimized_dataset))
optimized_dataset.info(memory_usage='deep')

# 8
# отобрать свои 10 колонок
need_column = dict()
column_names = ['SHORT_DESC', 'LOAD_TIME', 'STATE_ANSI',
                 'UTIL_PRACTICE_DESC', 'STATISTICCAT_DESC', 'DOMAIN_DESC',
                 'YEAR', 'VALUE', 'VALUE', 'CV_%']
opt_dtypes = optimized_dataset.dtypes
for key in dataset.columns:
    need_column[key] = opt_dtypes[key]
    print(f"{key}:{opt_dtypes[key]}")

with open("dtypes_2.json", mode="w") as file:
    dtype_json = need_column.copy()
    for key in dtype_json.keys():
        dtype_json[key] = str(dtype_json[key])

    json.dump(dtype_json, file)

# 9
read_and_optimized = pd.read_csv(file_name, usecols=lambda x: x in column_names, dtype=need_column)

C:\Users\Meow\AppData\Local\Temp\ipykernel_25252\3037962000.py:5: DtypeWarning: Columns (18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_name, compression="zip")


file in memory size =    4851751 КБ
SHORT_DESC                    :     297947 КБ:      6.141% : object
LOCATION_DESC                 :     239058 КБ:     4.9273% : object
UTIL_PRACTICE_DESC            :     228765 КБ:     4.7151% : object
DOMAINCAT_DESC                :     226164 КБ:     4.6615% : object
PRODN_PRACTICE_DESC           :     224489 КБ:      4.627% : object
LOAD_TIME                     :     212100 КБ:     4.3716% : object
CLASS_DESC                    :     195905 КБ:     4.0378% : object
COUNTRY_NAME                  :     193386 КБ:     3.9859% : object
GROUP_DESC                    :     191257 КБ:      3.942% : object
SECTOR_DESC                   :     186413 КБ:     3.8422% : object
DOMAIN_DESC                   :     183982 КБ:     3.7921% : object
COMMODITY_DESC                :     183302 КБ:     3.7781% : object
STATISTICCAT_DESC             :     181633 КБ:     3.7437% : object
STATE_NAME                    :     178156 КБ:      3.672% : object
ASD_DESC    